<a href="https://colab.research.google.com/github/andcappe/AndreaFrontieraEfficiente/blob/main/DOWNLOAD_DATA_FROM_YAHOO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
#############################################
# Installo le librerie necessarie           #
#############################################
!pip install pandas
!pip install pandas-datareader
!pip install yfinance==0.2.40


In [43]:
#############################################
# Importo le librerie e faccio un override  #
# di yfinance altrimenti non scarica i dati #
#############################################
import pandas as pd
from pandas_datareader import data as web
import matplotlib
from datetime import datetime, timedelta # Import the datetime class from the datetime module
#import datetime as datetime
import time
import yfinance as yf
yf.pdr_override()


In [44]:
#############################################
# Importo libreria e leggo un file excel    #
# con i dati su tre colonne                 #
# SIMBOLO DESCRIZIONE VALUTA                #
# creo un dataframe chiamato valid_df       #
#############################################
import os
try:

    cartella_dati = '/content/'
    nome_file = 'indici.xlsx'
    percorso_completo = os.path.join(cartella_dati, nome_file)
    valid_df = pd.read_excel(percorso_completo)
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")
    # You can add more error handling logic here, like asking the user for the correct path

# Nuova sezione

In [50]:
#############################################
# Elimino i NAN                             #
# elimino le colonne senza intestazione     #
#############################################
valid_df = valid_df.dropna()
invalid_cols = [i for i in valid_df.columns if i.startswith('Unnamed')]
        # Verifica colonne senza intestazione
valid_df =valid_df.drop(columns=invalid_cols)


# Nuova sezione

In [46]:

df_dict = valid_df.set_index(valid_df.columns[0]).to_dict()[valid_df.columns[1]]
#crea dizionario tikers descrizione


In [47]:
ticker_tup = list(df_dict.items())



In [48]:
#############################################
# Imposto data iniziale                     #
# Imposto data finale                       #
# Imposto una lista di SIMBOLI              #
# Imposto timeframe                         #
#############################################
end_date  = datetime.today()
start_date = end_date - timedelta(days=900)

temp = start_date
temp1 = end_date
temp3 = list(valid_df.get('SIMBOLO'))
temp2 ='1d'



In [32]:
#############################################
# Imposto data iniziale                     #
# Imposto data finale                       #
# Imposto una lista di SIMBOLI              #
# Imposto timeframe                         #
#############################################
#tup=(valid_df)



         SIMBOLO                DESCRIZIONE VALUTA
0        SUAS.MI           MSCI USA SRI USD    EUR
1        SUSW.MI      MSCI World SRI EUR MI    EUR
2   0P000026CD.L  PURISIMA TOTAL RETURN EUR    GBP
3           ^DJI             DJI INDUSTRIAL    USD
4           ^DJU                DJU UTILITY    USD
5       CSNDX.SW             ETF NASDAQ 100    USD
6          ^GSPC                     S&P500    USD
7            IWV           ETF RUSSELL 3000    USD
8            IWX   ETF RUSSELL TOP200 VALUE    USD
9            IWY  ETF RUSSELL TOP200 GROWTH    USD
10           IWB           ETF RUSSELL 1000    USD
11           IWD     ETF RUSSELL 1000 VALUE    USD
12           IWF     ETF RUSSELL 1000 GROWT    USD
13           IWM           ETF RUSSELL 2000    USD
14           IWN     ETF RUSSELL 2000 VALUE    USD
15           IWO     ETF RUSSELL 2000 GROWT    USD
16           IWR                ETF MID CAP    USD
17           IWS          ETF MID CAP VALUE    USD
18           IWP          ETF M

In [33]:
#############################################
# Devo verificare cosa faccia isin          #
# Ceo una tabella di informazioni           #
# dalla quale estrapoliamo le colonne       #
# aggiungiamo ai tickers le valute          #
# creo una lista tickers + valute           #
#############################################
df2 = valid_df[valid_df['SIMBOLO'].isin(temp3)]

simbol=df2['SIMBOLO']
# Creo una serie con i simboli
myvalues=df2['DESCRIZIONE']
# Creo una serie con la descrizione
val=df2['VALUTA']
#Creo una serie con le valute
valute =['EURUSD=X', 'EURGBP=X']
# Aggiungo i cambi da scaricare per convertire i valori in valuat
tick_val = simbol.tolist() + valute


In [34]:
#############################################
# Creo una funzione per scaricare dati      #
# Uso una libreria di yfinance              #
# con l'override la uso con pandas          #
# uso tickval tickers + valute per i dati   #
# scarico i dati e faccio interpolazione    #
# per eliminare i NAN                       #
#############################################
def get_web_data_user_input(tickers_to_use = '', startdate = '', enddate = '', interval = ''):

    try:

        data = web.get_data_yahoo(tickers_to_use, start = startdate,
                                   end = enddate, interval = interval, group_by='ticker')  #, interval = interval



        return data
    except Exception as err:
        return f'Error getting data from Yahoo finance {err}'

web_data_df = get_web_data_user_input(tickers_to_use = tick_val , startdate = temp , enddate = temp1, interval = temp2)
web_data_df=web_data_df.interpolate(method = 'linear')



[*********************100%%**********************]  32 of 32 completed


In [54]:
#############################################
# Aggiungo le colonne dell'EURO             #
# converto i prezzi dei titoli in euro      #
# elimino poi le colonne con le valute      #
#############################################
if tup.shape[1]==3:

            web_data_df['EUREUR=X', ('Open')]=1
            web_data_df['EUREUR=X', ('High')]=1
            web_data_df['EUREUR=X', ('Low')]=1
            web_data_df['EUREUR=X', ('Close')]=1
            web_data_df['EUREUR=X', ('Adj Close')]=1
            web_data_df['EUREUR=X', ('Volume')]=1
            # Aggiungo una colonna del cambio euro euro
            res = {simbol.tolist()[i]: val.tolist()[i] for i in range(len(simbol.tolist()))}
            # Creo un dizionario ticker simbolo : valuta
            for k,v in res.items():
                if v == 'USD':
                    web_data_df[(k, 'Adj Close')]=web_data_df[k]['Adj Close']/web_data_df['EURUSD=X']['Adj Close']

                elif v == 'GBP':
                    web_data_df[(k, 'Adj Close')]=web_data_df[k]['Adj Close']/web_data_df['EURGBP=X']['Adj Close']

                else:
                    web_data_df[(k, 'Adj Close')]=web_data_df[k]['Adj Close']/web_data_df['EUREUR=X']['Adj Close']
        # Applico i tassi di cambio
else:
     pass
web_data_df = web_data_df.get(temp3)


In [29]:
#############################################
# Esporto il file excel                     #
#############################################
percorso ="dati_matlab.xlsx"
web_data_df.to_excel(percorso, sheet_name="Dataset", index=True)